# Genentech – 404 Challenge


In [ ]:
!pip install hyperimpute

In [1]:
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

workspace = Path("workspace")
results_dir = Path("results")
data_dir = Path("data")

workspace.mkdir(parents=True, exist_ok=True)

warnings.filterwarnings("ignore")

# Consider a feature as categorical if there are at most <cat_limit> unique values for that feature
cat_limit = 10

## Load and augment data

For each csv:
1. Sort the dataset around the tuple __("RID_HASH", "VISCODE")__.
2. __Augment__ each row some temporal details: __total known visits__ and the __last known visit__.
3. Using a MinMaxScaler, scale the columns ["MMSE", "ADAS13", "Ventricles", "Hippocampus", "WholeBrain", "Entorhinal", "Fusiform", "MidTemp"]. While this initially was for the NN experiments, it also helped some linear models in the longitudinal imputation.

While the visits are not complete in the test set, adding the total visits and last known visit improved the public score.


----
Other failed approaches to augment the dataset include:
- Learn a latent space using an RNN, LSTM, or Transformer, - from each row and its missingness mask -  and append it to each row. 
- Add details related to trends from [Ventricles, Hippocampus,WholeBrain, Entorhinal, Fusiform, MidTemp].

While these approaches might help in a forecasting problem, they didn't improve the imputation error here. 

The latent space isn't trivial to model with missing values, and the neural net seemed to overfit the `dev_set,` leading to a poor score in the test data. The trends didn't help either, as the missingness mangled them. 

These additional experiments are not included here for clarity.

In [2]:
def augment_base_dataset(df):
    """For each patient(RID_HASH), add some information from the other visits,
    like the total known visits, and the last visit.
    """
    for rid in df["RID_HASH"].unique():
        visits = len(df[df["RID_HASH"] == rid])
        last_visit = df[df["RID_HASH"] == rid]["VISCODE"].max()

        df.loc[df["RID_HASH"] == rid, "total_visits"] = visits
        df.loc[df["RID_HASH"] == rid, "last_visit"] = last_visit

    return df


def dataframe_hash(df: pd.DataFrame) -> str:
    """Dataframe hashing, used for caching/backups"""
    cols = sorted(list(df.columns))
    return str(abs(pd.util.hash_pandas_object(df[cols].fillna(0)).sum()))

In [3]:
dev_set = pd.read_csv(data_dir / "dev_set.csv")
dev_set = dev_set.sort_values(["RID_HASH", "VISCODE"])
dev_set = augment_base_dataset(dev_set)

scaled_cols = [
    "MMSE",
    "ADAS13",
    "Ventricles",
    "Hippocampus",
    "WholeBrain",
    "Entorhinal",
    "Fusiform",
    "MidTemp",
]

scaler = MinMaxScaler().fit(dev_set[scaled_cols])
dev_set[scaled_cols] = scaler.transform(dev_set[scaled_cols])

dev_set

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
2163,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,0,79.1,0,20,1.0,1.0,0.5,0.923077,0.164384,0.071871,0.548646,0.376516,0.464021,0.194906,0.400709,2.0,6.0
154,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,6,79.6,0,20,1.0,1.0,1.5,0.923077,0.237397,0.071956,0.548307,0.366398,0.403880,0.193367,0.397291,2.0,6.0
1385,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,0,72.9,1,12,1.0,1.0,1.0,1.000000,0.123288,0.142655,0.525169,0.235599,0.513404,0.356253,0.294774,6.0,60.0
2698,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,6,73.4,1,12,1.0,1.0,1.0,1.000000,0.164384,0.144729,0.549210,0.230361,0.435097,0.322395,0.294175,6.0,60.0
2291,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,12,73.9,1,12,1.0,1.0,1.0,0.961538,0.109589,0.155550,0.527878,0.215944,0.487831,0.342600,0.277552,6.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,60,79.8,1,19,1.0,0.0,3.0,0.923077,0.223699,0.170895,0.357020,0.321346,0.310935,0.399047,0.461476,7.0,102.0
2646,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,102,83.3,1,19,1.0,0.0,3.0,0.846154,0.168904,0.178231,0.352043,0.309095,0.256790,0.372685,0.416478,7.0,102.0
1962,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,0,72.1,0,12,1.0,0.0,0.5,0.884615,0.150685,0.416382,0.602438,0.636654,0.610229,0.743037,0.624631,3.0,24.0
122,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,12,73.1,0,12,1.0,0.0,1.0,0.961538,0.155205,0.398451,0.608521,0.634650,0.617108,0.729087,0.638477,3.0,24.0


In [4]:
dev_1 = pd.read_csv(data_dir / "dev_1.csv")
dev_1 = dev_1.sort_values(["RID_HASH", "VISCODE"])
dev_1 = augment_base_dataset(dev_1)
dev_1[scaled_cols] = scaler.transform(dev_1[scaled_cols])

dev_1

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
2163,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,0,NaN,0.0,20.0,1.0,1.0,0.5,0.923077,0.164384,NaN,NaN,0.376516,NaN,NaN,NaN,2.0,6.0
154,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,6,79.6,0.0,20.0,1.0,1.0,1.5,0.923077,0.237397,0.071956,0.548307,0.366398,0.403880,0.193367,0.397291,2.0,6.0
1385,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,0,NaN,1.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,0.525169,0.235599,0.513404,0.356253,0.294774,6.0,60.0
2698,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,6,NaN,1.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,0.549210,0.230361,0.435097,0.322395,0.294175,6.0,60.0
2291,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,12,NaN,1.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,0.527878,0.215944,0.487831,0.342600,0.277552,6.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,60,79.8,1.0,19.0,NaN,0.0,NaN,NaN,NaN,0.170895,NaN,0.321346,NaN,NaN,NaN,7.0,102.0
2646,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,102,83.3,1.0,19.0,NaN,0.0,NaN,NaN,NaN,0.178231,NaN,0.309095,NaN,NaN,NaN,7.0,102.0
1962,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,0,72.1,NaN,12.0,1.0,0.0,0.5,0.884615,0.150685,0.416382,0.602438,NaN,0.610229,0.743037,0.624631,3.0,24.0
122,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,12,73.1,NaN,12.0,1.0,0.0,1.0,0.961538,0.155205,0.398451,0.608521,NaN,0.617108,0.729087,0.638477,3.0,24.0


In [5]:
dev_2 = pd.read_csv(data_dir / "dev_2.csv")
dev_2 = dev_2.sort_values(["RID_HASH", "VISCODE"])
dev_2 = augment_base_dataset(dev_2)
dev_2[scaled_cols] = scaler.transform(dev_2[scaled_cols])

dev_2

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
2163,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,0,79.1,0.0,20.0,1.0,1.0,0.5,0.923077,0.164384,0.071871,0.548646,0.376516,0.464021,0.194906,0.400709,2.0,6.0
154,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,6,79.6,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.071956,0.548307,NaN,0.403880,0.193367,0.397291,2.0,6.0
1385,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,0,72.9,NaN,12.0,1.0,1.0,1.0,1.000000,0.123288,0.142655,0.525169,NaN,0.513404,0.356253,0.294774,6.0,60.0
2698,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,6,NaN,NaN,12.0,1.0,1.0,1.0,1.000000,0.164384,NaN,0.549210,NaN,0.435097,0.322395,0.294175,6.0,60.0
2291,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,12,NaN,NaN,12.0,1.0,1.0,1.0,0.961538,0.109589,NaN,NaN,NaN,NaN,NaN,NaN,6.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,60,NaN,NaN,19.0,1.0,0.0,3.0,0.923077,0.223699,NaN,0.357020,NaN,0.310935,0.399047,0.461476,7.0,102.0
2646,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,102,NaN,NaN,19.0,1.0,0.0,3.0,0.846154,0.168904,NaN,0.352043,NaN,0.256790,0.372685,0.416478,7.0,102.0
1962,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,0,72.1,NaN,12.0,NaN,0.0,NaN,NaN,NaN,0.416382,0.602438,NaN,0.610229,0.743037,0.624631,3.0,24.0
122,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,12,NaN,NaN,12.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,24.0


In [6]:
test_A = pd.read_csv(data_dir / "test_A.csv")
test_A = augment_base_dataset(test_A)
test_A[scaled_cols] = scaler.transform(test_A[scaled_cols])

test_A

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
0,988b6137f4352c01e4b52790505caa0c3ec438f117000a...,24,NaN,NaN,18.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,36.0
1,fb640cef87a6af00053e632140ce18f5722431bb92576b...,12,66.4,1.0,18.0,1.0,1.0,1.5,0.961538,0.077671,0.145063,NaN,0.542904,NaN,NaN,NaN,5.0,24.0
2,f24f78d62c90319b575dfb48a482159c4d0df14cb71530...,66,74.5,0.0,14.0,0.0,0.0,0.0,0.961538,0.050274,0.559104,0.565102,0.753302,0.641093,0.911086,0.866886,5.0,96.0
3,da4cbd3f09e8ddc87cc72e542d43f072e7df288face65e...,0,NaN,NaN,16.0,0.0,0.0,0.0,1.000000,0.191781,NaN,NaN,NaN,NaN,NaN,NaN,2.0,36.0
4,f665c6ee86356bdd135be03c61348607cabd64ed8433ba...,12,82.7,NaN,13.0,NaN,1.0,NaN,NaN,NaN,0.206872,0.353047,NaN,0.208289,0.188006,0.363489,7.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,51923c5d7573ef46aa9197cae78c3305abea5b3479331f...,6,83.5,NaN,18.0,1.0,0.0,2.5,0.846154,0.150685,0.252103,0.541648,NaN,0.454850,0.480663,0.479467,5.0,60.0
1324,06407d9ec85d62cd38189108ddffec23822f421b3db357...,0,NaN,NaN,20.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
1325,e5015703a58ccd5582a46d9f4a779edf062d683f3ae873...,132,83.0,0.0,20.0,0.0,1.0,0.0,0.961538,0.205479,0.193712,0.533115,0.491958,0.513933,0.508464,0.573437,9.0,132.0
1326,cf6ea2601bb119113371df79931cc3734b77218f734ad0...,12,82.4,NaN,18.0,1.0,0.0,0.5,0.884615,0.205479,0.272259,0.572799,NaN,0.597707,0.380480,0.361608,3.0,12.0


In [7]:
test_B = pd.read_csv(data_dir / "test_B.csv")
test_B = augment_base_dataset(test_B)
test_B[scaled_cols] = scaler.transform(test_B[scaled_cols])

test_B

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
0,90a4f1869cf459af5fe39e53f1c328540f1dcf5a1908f7...,60,67.9,1.0,20.0,0.0,1.0,0.0,1.000000,0.123288,0.069404,NaN,0.330562,NaN,NaN,NaN,6.0,60.0
1,fad8ca8f903cf3ddf566926eabdb8718e8568962675519...,30,69.1,NaN,16.0,0.0,0.0,0.0,0.961538,0.059315,0.162418,0.749086,NaN,0.899471,0.724619,0.481817,3.0,30.0
2,d342fb7689e49c754709870c77e1aa3ed770dd193e9f9c...,12,NaN,NaN,12.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12.0
3,5319e7ba149f0f81715b5e7f854036fc937141840bbd52...,6,NaN,NaN,18.0,NaN,0.0,NaN,NaN,NaN,NaN,0.476637,NaN,0.430159,0.520727,0.415281,5.0,126.0
4,6eef135d8c4eca67b0e130b8f4aedbc37a99938224d661...,0,NaN,0.0,16.0,0.0,0.0,0.0,0.961538,0.095890,NaN,0.605756,0.508509,0.629277,0.604379,0.449383,5.0,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,fbf6267bf7d92b507feb4957d7aa90ea5bb50893bb79d4...,0,NaN,NaN,12.0,NaN,2.0,NaN,NaN,NaN,NaN,0.492325,NaN,0.497354,0.374770,0.460151,7.0,60.0
1466,03e8ddc654f8e27332c5b09618b355d7f9529d614adb0f...,12,81.5,0.0,15.0,NaN,0.0,NaN,NaN,NaN,0.264134,NaN,0.208488,NaN,NaN,NaN,3.0,12.0
1467,1156748dfd6e69e1f364c31584e957d3b1ef656b898942...,0,NaN,0.0,18.0,0.0,0.0,0.0,0.961538,0.136986,NaN,NaN,0.686284,NaN,NaN,NaN,4.0,84.0
1468,0c7e17c442e715e067bd472c1e472b4937914d7fb8d492...,12,NaN,NaN,16.0,NaN,0.0,NaN,NaN,NaN,NaN,0.553883,NaN,0.753086,0.606563,0.458784,4.0,42.0


In [8]:
submission = pd.read_csv(data_dir / "sample_submission.csv")

submission.values[1]

array(['6b6a7136f42a8dbd469a201b88e2abb54a93667822761357db2f6d620da6af8a_0_Ventricles_test_A',
       40613.0818580834], dtype=object)

## EDA

__Observations__:
- While a time series problem, several patients have a single visit(255 in the training set), which must be handled as a static imputation problem.
- Features [`RID_HASH`, `PTGENDER_num`, `PTEDUCAT`, `APOE4`] are constant by the patient and need special handling - we should not have multiple values by  `RID_HASH`.
- There is a correlation between `VISCODE` and `AGE`, which must be respected. More precisely, 1 `VISCODE` step maps to 1 month. `VISCODE` = 6 maps to 6 months after the baseline visit.
- `CDRSB` is a multiple of 0.5
- `DX_NUM` and `MMSE` must be integers
- `ADAS13` is multiple of 1/3.

In [9]:
def get_visit_counts(df):
    """Get the number of patients for each visit count, eg. patients with single visits, 2 visits etc."""
    out = df.groupby(["RID_HASH"]).count()["VISCODE"].value_counts()
    out.index = out.index.rename("Visits")
    out.name = "Patients count by visit count"

    return out


# test_A
# get_visit_counts(test_A)

# test_B
# get_visit_counts(test_B)

# Train data
get_visit_counts(dev_set)

Visits
2     289
1     255
3     206
4     163
5     128
6      68
7      45
8      33
9      23
10     13
11      2
12      1
Name: Patients count by visit count, dtype: int64

In [10]:
def get_constants_by_patients(df):
    """Get constant feature by patient."""
    candidates = list(df.columns)
    for rid in df["RID_HASH"].unique():
        patient = df[df["RID_HASH"] == rid]
        diffs = patient.drop(columns=["RID_HASH"]).diff().fillna(0)

        zeros = (diffs == 0).sum()

        for col in zeros.index:
            if zeros[col] != len(patient):
                if col in candidates:
                    candidates.remove(col)

    return candidates


get_constants_by_patients(dev_set)
# get_constants_by_patients(test_A)
# get_constants_by_patients(test_B)

['RID_HASH', 'PTGENDER_num', 'PTEDUCAT', 'APOE4', 'total_visits', 'last_visit']

In [11]:
def test_age_viscode_corr(df):
    """Test AGE - VISCODE relation."""
    candidates = list(df.columns)
    for rid in df["RID_HASH"].unique():
        patient = df[df["RID_HASH"] == rid]
        age_diffs = patient["AGE"].diff().fillna(0).values
        viscode_diffs = patient["VISCODE"].diff().fillna(0).values

        assert np.allclose(12 * age_diffs, viscode_diffs)
    print("All good. One VISCODE step maps to 1 month")


test_age_viscode_corr(dev_set)

All good. One VISCODE step maps to 1 month


## Iterative Horizontal(instance-wise) imputation

We first define a method for instance-wise imputation: for a single patient visit, to impute the missing values from the observed values. 

This is especially valuable for patients with single visits, imputing static features(`PTGENDER`, `PTEDUCAT`), but also to seed the longitudinal imputation.

For the task, we are using [__HyperImpute__](https://github.com/vanderschaarlab/hyperimpute), an iterative imputation algorithm, which generalizes MICE and missForest, by allowing any type of base learner(not just linear/random forest), and which uses AutoML to tune the models by column and by iteration.

The pool of __base learners__ is: 
 - classifiers: [`xgboost`, `catboost`, `logistic_regression`, `random_forest`, `lgbm`]
 - regressors: [`xgboost_regressor`, `catboost_regressor`, `linear_regression`, `random_forest_regressor`, `lgbm_regressor`]

For selecting a classifier, the internal optimizer searches for the maximal AUCROC score obtained on the observed data(without any missing values). For selecting a regressor, the optimizer searches for the maximal R2 score obtained on the observed data.

#### Model search
The models are re-evaluated for each column and for for each imputation iteration. 

The following table contains a snippet from the test set from the first imputation iteration.
For each target, we list the selected model to impute that column and its arguments.

The evaluation score is the `AUCROC` for classifiers(with categorical targets), and `R2` for regressors.
The evaluation is performed on observed data, and for each target column, we use the rest of the features from each patient. For example, for target "AGE", we benchmark a model trained on the patient features without the "AGE" column.




| Target Column | Score on the observed data | Model   | Model parameters                                                                                                                                                                                                                                                                                                                           |
|---------------|---------------------------------|---------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| PTGENDER_num  | 0.9581                          | xgboost | { 	'reg_lambda': 5.047146750787419, 	'reg_alpha': 6.76511248668679, 	'colsample_bytree': 0.8104392038078263, 	'colsample_bynode': 0.8362908590082906, 	'colsample_bylevel': 0.8415650566586329, 	'subsample': 0.899881421386532, 	'lr': 0.001, 	'max_depth': 5, 	'n_estimators': 282, 	'min_child_weight': 6, 	'max_bin': 384, 	'grow_policy': 1 }     |
| MidTemp       | 0.8257                          | xgboost_regressor | {'reg_lambda': 2.849409466197371, 'reg_alpha': 0.12666464008400558, 'lr': 0.001, 'colsample_bytree': 0.8435311327845306, 'colsample_bynode': 0.7146987739735369, 'colsample_bylevel': 0.6200355491527186, 'subsample': 0.5832358516941026, 'max_depth': 5, 'n_estimators': 299, 'min_child_weight': 7, 'max_bin': 354, 'grow_policy': 0}   |
| PTEDUCAT      | 0.5983                          | xgboost_regressor | {'reg_lambda': 1.2043879741664179, 'reg_alpha': 0.43791636068939466, 'lr': 0.0001, 'colsample_bytree': 0.8787783651242453, 'colsample_bynode': 0.5585583183603081, 'colsample_bylevel': 0.8510294755051252, 'subsample': 0.8979610242164071, 'max_depth': 5, 'n_estimators': 295, 'min_child_weight': 5, 'max_bin': 435, 'grow_policy': 0} |
| Hippocampus   | 0.8485                          | xgboost_regressor | {'reg_lambda': 4.089278739805318, 'reg_alpha': 0.17543671290431134, 'lr': 0.001, 'colsample_bytree': 0.8786337139809749, 'colsample_bynode': 0.8318946749900471, 'colsample_bylevel': 0.4180573427733283, 'subsample': 0.8994410403323976, 'max_depth': 5, 'n_estimators': 300, 'min_child_weight': 51, 'max_bin': 279, 'grow_policy': 0}  |
| AGE           | 0.7392                          | xgboost_regressor | {'reg_lambda': 2.8968259886307823, 'reg_alpha': 2.689625820088316, 'lr': 0.0001, 'colsample_bytree': 0.7833585243779915, 'colsample_bynode': 0.7816022024981492, 'colsample_bylevel': 0.8547872455663319, 'subsample': 0.8598089876739611, 'max_depth': 5, 'n_estimators': 300, 'min_child_weight': 10, 'max_bin': 310, 'grow_policy': 1}  |
| Ventricles    | 0.6632                          | xgboost_regressor | {'reg_lambda': 5.967189083549597, 'reg_alpha': 0.2808968422111205, 'lr': 0.01, 'colsample_bytree': 0.5217102570325618, 'colsample_bynode': 0.8804696194235598, 'colsample_bylevel': 0.8708840451006549, 'subsample': 0.8697548865770208, 'max_depth': 5, 'n_estimators': 274, 'min_child_weight': 15, 'max_bin': 476, 'grow_policy': 1}    |
| APOE4         | 0.9273                          | xgboost | {'reg_lambda': 2.8690526199504927, 'reg_alpha': 3.522038047599354, 'colsample_bytree': 0.8424641257249423, 'colsample_bynode': 0.8665271329717139, 'colsample_bylevel': 0.5663182904186024, 'subsample': 0.7565978995769022, 'lr': 0.01, 'max_depth': 4, 'n_estimators': 204, 'min_child_weight': 3, 'max_bin': 276, 'grow_policy': 0}     |
| WholeBrain    | 0.8409                          | xgboost_regressor | {'reg_lambda': 7.307980134321348, 'reg_alpha': 0.030638315290805348, 'lr': 0.001, 'colsample_bytree': 0.7045048629753644, 'colsample_bynode': 0.6717575448956548, 'colsample_bylevel': 0.8523579198631583, 'subsample': 0.811683946029976, 'max_depth': 4, 'n_estimators': 263, 'min_child_weight': 9, 'max_bin': 283, 'grow_policy': 1}   |
| CDRSB         | 0.8647                          | xgboost_regressor | {'reg_lambda': 5.7188353890043215, 'reg_alpha': 1.5070276030155896, 'lr': 0.001, 'colsample_bytree': 0.8568652904045923, 'colsample_bynode': 0.8228490141313873, 'colsample_bylevel': 0.717485389358915, 'subsample': 0.838052956971084, 'max_depth': 5, 'n_estimators': 203, 'min_child_weight': 0, 'max_bin': 421, 'grow_policy': 0}     |
| MMSE          | 0.7849                          | xgboost_regressor | {'reg_lambda': 4.411307527948391, 'reg_alpha': 0.31020731746278507, 'lr': 0.001, 'colsample_bytree': 0.8053955379772784, 'colsample_bynode': 0.6706019993671946, 'colsample_bylevel': 0.7185383704960561, 'subsample': 0.5333272846743093, 'max_depth': 5, 'n_estimators': 97, 'min_child_weight': 0, 'max_bin': 451, 'grow_policy': 1}    |
| ADAS13        | 0.8387                          | xgboost_regressor | {'reg_lambda': 7.057171358250669, 'reg_alpha': 0.018439155294796027, 'lr': 0.001, 'colsample_bytree': 0.7706572341235782, 'colsample_bynode': 0.7583265692412863, 'colsample_bylevel': 0.6436305828182355, 'subsample': 0.6700622116024618, 'max_depth': 4, 'n_estimators': 276, 'min_child_weight': 12, 'max_bin': 341, 'grow_policy': 0} |
| DX_num        | 0.9901                          | xgboost | {'reg_lambda': 3.491083476792939, 'reg_alpha': 0.4780607781520152, 'colsample_bytree': 0.7135088508567039, 'colsample_bynode': 0.7401906044406585, 'colsample_bylevel': 0.877133448906174, 'subsample': 0.6640341453442069, 'lr': 0.01, 'max_depth': 5, 'n_estimators': 284, 'min_child_weight': 0, 'max_bin': 476, 'grow_policy': 1}      |
| Fusiform      | 0.7955                          | xgboost_regressor | {'reg_lambda': 7.283324221523343, 'reg_alpha': 0.02239137521568263, 'lr': 0.0001, 'colsample_bytree': 0.8142461647714889, 'colsample_bynode': 0.4828281981820367, 'colsample_bylevel': 0.8124281636754882, 'subsample': 0.5836904623813199, 'max_depth': 5, 'n_estimators': 300, 'min_child_weight': 8, 'max_bin': 451, 'grow_policy': 1}  |
| Entorhinal    | 0.7384               | xgboost_regressor | {'reg_lambda': 8.488614432064912, 'reg_alpha': 0.21197074583337316, 'lr': 0.001, 'colsample_bytree': 0.8787074288786586, 'colsample_bynode': 0.8439109829757719, 'colsample_bylevel': 0.43614024854551137, 'subsample': 0.7445117791091113, 'max_depth': 5, 'n_estimators': 282, 'min_child_weight': 55, 'max_bin': 266, 'grow_policy': 1} |

In [12]:
import hyperimpute.logger as log
from hyperimpute.plugins.imputers import Imputers

log.remove()
log.add(sink=sys.stderr, level="INFO")


def horizontal_imputation(train_data, test_data):
    imputed_test_data = test_data.copy()
    imputer_kwargs = {
        "optimizer": "bayesian",
        "classifier_seed": [
            "xgboost",
            "catboost",
            "logistic_regression",
            "random_forest",
        ],
        "regression_seed": [
            "xgboost_regressor",
            "catboost_regressor",
            "linear_regression",
            "random_forest_regressor",
        ],
        "select_lazy": False,
        "optimize_thresh": 10000,
        "class_threshold": cat_limit,
    }

    imputer = Imputers().get(
        "hyperimpute",
        **imputer_kwargs,
    )
    imputation_input = pd.concat([train_data, test_data], ignore_index=True)
    imputation_ids = imputation_input["RID_HASH"]
    imputation_input = imputation_input.drop(columns=["RID_HASH"])

    imputed_test_data = imputer.fit_transform(imputation_input)
    imputed_test_data = imputed_test_data.tail(len(test_data))

    out_cols = ["RID_HASH"] + list(imputed_test_data.columns)
    imputed_test_data["RID_HASH"] = test_data["RID_HASH"].values

    return imputed_test_data[out_cols]

## Iterative Longitudinal imputation

It is important for patients with multiple visits to impute constrained by other visits.

`HyperImpute` is dedicated to static imputation, and cannot be used directly for this task.
However, we can use its base learners for this task by preprocessing the data.

#### Preprocessing
Two sets of estimators are trained, __forward__ and __reverse__, for the direction in which we try to approximate a particular feature.

For every target feature:
- We retrieve the previous visit, given the direction we work in. For the direction __forward__, the previous visit is the previous `VISCODE`. For the direction __reverse__, the previous visit is the next `VISCODE`. 
- We generate a training set consisting of the previous visit, and the current visit without the target feature
- The `prepare_temporal_data` function implements this strategy.

#### Model Search
The search pool consists of "XGBoost", "CatBoost", "LGBM", "Random forest", "KNearestNeighbor" and linear models.

Similar to the Horizontal imputation setup, we run the AutoML logic on the preprocessed data, selecting the optimal AUCROC for classifiers and the optimal R2 score for regressors.

The static features - `PTEDUCAT`, `PTGENDER`, `APOE4`, and the ones correlated with `VISCODE`(`AGE`), are not evaluted here.


Selected Predictors for the __forward__ direction

| Target Column | Score on the observed data | Model             | Model parameters                                                                                                                                                                                                                                                                                                                    |
|---------------|---------------------------------|-------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| MidTemp       | 0.8511                          | ridge_regression | {}                                                                                                                                                                                                                                                                                                                                  |
| Hippocampus   | 0.8693                          | xgboost_regressor | {'reg_lambda': 1.664443906152452, 'reg_alpha': 0.11216371454959836, 'max_depth': 4, 'n_estimators': 59, 'lr': 0.0001}                                                                                                                                                                                                               |
| Ventricles    | 0.7916                          | ridge_regression |  {'max_iter': 100, 'solver': 1} # HyperImpute's defaults                                                                                                                                                                                                                                                                                                        |
| WholeBrain    | 0.89219                         | ridge_regression | {'max_iter': 10000, 'solver': 1}                                                                                                                                                                                                                                                                                                                                  |
| CDRSB         | 0.8235                          | ridge_regression | {'max_iter': 10000, 'solver': 1}                                                                                                                                                                                                                                                                                                                                  |
| MMSE          | 0.7464                          | xgboost_regressor | {'reg_lambda': 2.1888371329742604, 'reg_alpha': 0.9312430343555846, 'max_depth': 2, 'n_estimators': 82, 'lr': 0.01}                                                                                                                                                                                                                 |
| ADAS13        | 0.8184                          | ridge_regression | {}                                                                                                                                                                                                                                                                                                       |
| DX_num        | 0.9877                          | xgboost           | {'reg_lambda': 8.962330859372681, 'reg_alpha': 8.608303558298164, 'colsample_bytree': 0.7745875968255177, 'colsample_bynode': 0.3990200802902815, 'colsample_bylevel': 0.8924327091878707, 'subsample': 0.8411533780703977, 'lr': 0.0001, 'max_depth': 3, 'n_estimators': 169, 'min_child_weight': 1, 'max_bin': 256, 'booster': 1} |
| Fusiform      | 0.8196                          | ridge_regression | {'max_iter': 10000, 'solver': 1}                                                                                                                                                                                                                                                                                                                                  |
| Entorhinal    | 0.71296                         | ridge_regression | {}                                                                                                                                                                                                                                                                                                                                  |


Selected Predictors selected for the __reverse__ direction

| Target Column | Score on the observed data | Model             | Model parameters                                                                                                                                                                                                                                                                                                                     |
|---------------|---------------------------------|-------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| MidTemp       | 0.8541                          | ridge_regression | {'max_iter': 1000, 'solver': 1}                                                                                                                                                                                                                                                                                                                                   |
| Hippocampus   | 0.8697                          | xgboost_regressor | {'reg_lambda': 0.8716130833968974, 'reg_alpha': 0.6316607826045982, 'max_depth': 4, 'n_estimators': 100, 'lr': 0.0001}                                                                                                                                                                                                               |
| Ventricles    | 0.7414                          | ridge_regression | {'max_iter': 100, 'solver': 1}                                                                                                                                                                                                                                                                                                                                   |
| WholeBrain    | 0.8917                          | ridge_regression | {'max_iter': 100, 'solver': 1}                                                                                                                                                                                                                                                                                                                                   |
| CDRSB         | 0.8143                          | xgboost_regressor | {'reg_lambda': 4.0066164445274195, 'reg_alpha': 5.238168400540354, 'max_depth': 2, 'n_estimators': 86, 'lr': 0.0001}                                                                                                                                                                                                                 |
| MMSE          | 0.7416                          | xgboost_regressor | {'reg_lambda': 6.668953586033021, 'reg_alpha': 0.690722830428056, 'max_depth': 2, 'n_estimators': 75, 'lr': 0.01}                                                                                                                                                                                                                    |
| ADAS13        | 0.8200                          | xgboost_regressor | {'reg_lambda': 3.3543843807030633, 'reg_alpha': 0.5623126283166717, 'max_depth': 3, 'n_estimators': 78, 'lr': 0.0001}                                                                                                                                                                                                                |
| DX_num        | 0.9834                          | xgboost           | {'reg_lambda': 9.581894273376596, 'reg_alpha': 3.996895430038997, 'colsample_bytree': 0.41805003542171487, 'colsample_bynode': 0.8056647613216464, 'colsample_bylevel': 0.6632455401989068, 'subsample': 0.7898718692552386, 'lr': 0.0001, 'max_depth': 5, 'n_estimators': 224, 'min_child_weight': 0, 'max_bin': 503, 'booster': 1} |
| Fusiform      | 0.8207                          | ridge_regression | {'max_iter': 1000, 'solver': 1}                                                                                                                                                                                                                                                                                                                                   |
| Entorhinal    | 0.7069                          | ridge_regression | {'max_iter': 100, 'solver': 1}                                                                                                                                                                                                                                                                                                                                   |


In [13]:
import numpy as np
import optuna
from hyperimpute.plugins.prediction import Classifiers, Regression
from hyperimpute.utils.optimizer import EarlyStoppingExceeded, create_study
from hyperimpute.utils.tester import evaluate_estimator, evaluate_regression
from sklearn.preprocessing import LabelEncoder

train_cols = list(dev_set.drop(columns=["RID_HASH"]).columns)

eval_cols = [
    "DX_num",
    "CDRSB",
    "MMSE",
    "ADAS13",
    "Ventricles",
    "Hippocampus",
    "WholeBrain",
    "Entorhinal",
    "Fusiform",
    "MidTemp",
]


def prepare_temporal_data(data, target_col: str, direction: str):
    target_train_data = []
    target_train_labels = []

    for item in data.groupby("RID_HASH"):
        local = item[1]
        local = local.sort_values(["RID_HASH", "VISCODE"])

        rid = local["RID_HASH"]

        prev_cols = [f"prev_{col}" for col in train_cols]
        prev_row = np.zeros(len(prev_cols))

        if direction == "forward":
            rows = local.iterrows()
        else:
            rows = local.iloc[::-1].iterrows()

        for idx, row in rows:
            target_val = row[target_col]
            tmp_row = row[train_cols].copy()
            src_data = tmp_row.to_frame().T.drop(columns=[target_col])

            src_data[prev_cols] = prev_row

            prev_row = tmp_row

            target_train_data.append(src_data)
            target_train_labels.append(target_val)

    target_train_data = pd.concat(target_train_data, ignore_index=True).astype(float)

    return target_train_data, target_train_labels


def evaluate_target(data, target_col: str, direction: str):
    train_data, labels = prepare_temporal_data(data, target_col, direction)
    assert target_col not in train_data.columns

    def evaluate_clf(plugin, args={}):
        model = plugin(**args)
        encoded_labels = LabelEncoder().fit_transform(labels)

        return evaluate_estimator(model, train_data, pd.Series(encoded_labels))["clf"][
            "aucroc"
        ][0]

    def evaluate_reg(plugin, args={}):
        model = plugin(**args)
        return evaluate_regression(model, train_data, labels)["clf"]["r2"][0]

    best_score = -99
    best_target_plugin = None
    for (clf_type, reg_type) in [
        ("lgbm", "lgbm_regressor"),
        ("xgboost", "xgboost_regressor"),
        ("logistic_regression", "linear_regression"),
        ("catboost", "catboost_regressor"),
        ("random_forest", "random_forest_regressor"),
        ("kneighbors", "kneighbors_regressor"),
    ]:
        if len(np.unique(labels)) < cat_limit:
            if clf_type is None:
                continue
            plugin = Classifiers().get_type(clf_type)
            cbk = evaluate_clf
        else:
            if reg_type is None:
                continue
            plugin = Regression().get_type(reg_type)
            cbk = evaluate_reg

        study, pruner = create_study(
            study_name=f"long_imputation_{plugin.name()}_{target_col}_{direction}",
            direction="maximize",
            load_if_exists=True,
        )

        def objective(trial: optuna.Trial) -> float:
            args = plugin.sample_hyperparameters(trial)
            pruner.check_trial(trial)

            try:
                score = cbk(plugin, args)
            except BaseException:
                print("      failed evaluation", plugin.name(), args)
                return -5

            # print(f"    >>  {plugin.name()} {args} -> {score}")
            pruner.report_score(score)

            return score

        try:
            study.optimize(objective, n_trials=100, timeout=60 * 10)
        except EarlyStoppingExceeded:
            pass

        baseline_score = cbk(plugin)

        if study.best_value > baseline_score:
            score = study.best_value
            args = study.best_trial.params
        else:
            score = baseline_score
            args = {}

        if score > best_score:
            best_score = score
            best_target_plugin, best_target_plugin_args = plugin, args

    print(
        f"     >> Selected {target_col} --> {best_target_plugin.name()} -- {best_target_plugin_args}",
        best_score,
    )

    model = best_target_plugin(**best_target_plugin_args)

    if len(np.unique(labels)) < cat_limit:
        labels = LabelEncoder().fit_transform(labels)

    model.fit(train_data, labels)

    return model


def prepare_longitudinal_imputers(data, columns):
    imputers = {}

    for direction in ["forward", "reverse"]:
        imputers[direction] = {}
        for target_col in columns:
            model = evaluate_target(data, target_col, direction=direction)

            imputers[direction][target_col] = model

    return imputers

In [14]:
# for col in eval_cols:
#     evaluate_target(dev_set, col, direction = "forward")

#      >> Selected DX_num --> xgboost -- {'reg_lambda': 8.962330859372681, 'reg_alpha': 8.608303558298164, 'colsample_bytree': 0.7745875968255177, 'colsample_bynode': 0.3990200802902815, 'colsample_bylevel': 0.8924327091878707, 'subsample': 0.8411533780703977, 'lr': 0.0001, 'max_depth': 3, 'n_estimators': 169, 'min_child_weight': 1, 'max_bin': 256, 'booster': 1} 0.9877047491583673
#      >> Selected CDRSB --> linear_regression -- {'max_iter': 10000, 'solver': 1} 0.8235423012308886
#      >> Selected MMSE --> xgboost_regressor -- {'reg_lambda': 2.1888371329742604, 'reg_alpha': 0.9312430343555846, 'max_depth': 2, 'n_estimators': 82, 'lr': 0.01} 0.7464127570286078
#      >> Selected ADAS13 --> linear_regression -- {} 0.8184916603842121
#      >> Selected Ventricles --> linear_regression -- {'max_iter': 100, 'solver': 1} 0.7916836168497275
#      >> Selected Hippocampus --> xgboost_regressor -- {'reg_lambda': 1.664443906152452, 'reg_alpha': 0.11216371454959836, 'max_depth': 4, 'n_estimators': 59, 'lr': 0.0001} 0.8693374929967305
#      >> Selected WholeBrain --> linear_regression -- {'max_iter': 10000, 'solver': 1} 0.892197332159023
#      >> Selected Entorhinal --> linear_regression -- {} 0.7129676806181259
#      >> Selected Fusiform --> linear_regression -- {'max_iter': 10000, 'solver': 1} 0.8196491882018542
#      >> Selected MidTemp --> linear_regression -- {} 0.8511675281807548

In [15]:
# for col in eval_cols:
#     evaluate_target(dev_set, col, direction = "reverse")


#      >> Selected DX_num --> xgboost -- {'reg_lambda': 9.581894273376596, 'reg_alpha': 3.996895430038997, 'colsample_bytree': 0.41805003542171487, 'colsample_bynode': 0.8056647613216464, 'colsample_bylevel': 0.6632455401989068, 'subsample': 0.7898718692552386, 'lr': 0.0001, 'max_depth': 5, 'n_estimators': 224, 'min_child_weight': 0, 'max_bin': 503, 'booster': 1} 0.9834325918687735
#      >> Selected CDRSB --> xgboost_regressor -- {'reg_lambda': 4.0066164445274195, 'reg_alpha': 5.238168400540354, 'max_depth': 2, 'n_estimators': 86, 'lr': 0.0001} 0.8143407597683612
#      >> Selected MMSE --> xgboost_regressor -- {'reg_lambda': 6.668953586033021, 'reg_alpha': 0.690722830428056, 'max_depth': 2, 'n_estimators': 75, 'lr': 0.01} 0.7416617255780215
#      >> Selected ADAS13 --> xgboost_regressor -- {'reg_lambda': 3.3543843807030633, 'reg_alpha': 0.5623126283166717, 'max_depth': 3, 'n_estimators': 78, 'lr': 0.0001} 0.8200312184647823
#      >> Selected Ventricles --> linear_regression -- {'max_iter': 100, 'solver': 1} 0.7414274643766964
#      >> Selected Hippocampus --> xgboost_regressor -- {'reg_lambda': 0.8716130833968974, 'reg_alpha': 0.6316607826045982, 'max_depth': 4, 'n_estimators': 100, 'lr': 0.0001} 0.8697211565809551
#      >> Selected WholeBrain --> linear_regression -- {'max_iter': 100, 'solver': 1} 0.8917612746142106
#      >> Selected Entorhinal --> linear_regression -- {'max_iter': 100, 'solver': 1} 0.7069076933758623
#      >> Selected Fusiform --> linear_regression -- {'max_iter': 1000, 'solver': 1} 0.8207026292436043
#      >> Selected MidTemp --> linear_regression -- {'max_iter': 1000, 'solver': 1} 0.8541884735250632

In [16]:
from hyperimpute.utils.serialization import (load_model_from_file,
                                             save_model_to_file)

dev_set_id = dataframe_hash(dev_set)

imputers_bkp_file = (
    workspace
    / f"longitudinal_imputers_scaled_cat{cat_limit}_{dev_set_id}_automl_100.bkp"
)
if imputers_bkp_file.exists():
    longitudinal_imputers = load_model_from_file(imputers_bkp_file)
else:
    longitudinal_imputers = prepare_longitudinal_imputers(dev_set, eval_cols)
    save_model_to_file(imputers_bkp_file, longitudinal_imputers)

## Static features imputation

For the static features: [`PTGENDER_num`, `PTEDUCAT`, `APOE4`], we propagate the existing values for a patient to all the time points.

For the `AGE` feature, we propagate the value using the "VISCODE" value. For observations `i` and `i + 1` for a patient, we use the formula
```AGE[i + 1] = (VISCODE[i + 1] - VISCODE[i]) / 12 + AGE[i]```

In [17]:
const_by_patient = ["PTGENDER_num", "PTEDUCAT", "APOE4"]


def impute_consts(train_data, test_data):
    """For each patient, we fill the existing constant features to all time points."""
    test_data = test_data.copy()
    train_data = train_data.copy()

    train_data = train_data.sort_values(["RID_HASH", "VISCODE"])
    test_data = test_data.sort_values(["RID_HASH", "VISCODE"])

    for item in test_data.groupby("RID_HASH"):
        local = item[1]

        # fill consts
        for col in const_by_patient:
            if len(local[col].unique()) == 1:
                continue
            rid = local["RID_HASH"].unique()[0]

            val = local[col][~local[col].isna()].unique()[0]
            local[col] = local[col].fillna(val)
            test_data.loc[test_data["RID_HASH"] == rid, col] = test_data[
                test_data["RID_HASH"] == rid
            ][col].fillna(val)
            assert len(local[col].unique()) == 1, col

    return test_data


def impute_age(train_data, test_data):
    """AGE imputation logic for each patient, based on VISCODE and the existing "AGE" values
    For observations i and i + 1 for a patient,
    AGE[i + 1] = (VISCODE[i + 1] - VISCODE[i]) / 12 + AGE[i]
    """
    test_data = test_data.copy()
    train_data = train_data.copy()

    train_data = train_data.sort_values(["RID_HASH", "VISCODE"])
    test_data = test_data.sort_values(["RID_HASH", "VISCODE"])

    col = "AGE"

    for rid in test_data["RID_HASH"].unique():
        local = test_data[test_data["RID_HASH"] == rid]

        # fill age
        ages = local["AGE"]
        if ages.isna().sum() == 0:
            continue

        if ages.isna().sum() == len(ages):
            continue

        # forward impute age
        prev_viscode = 0
        prev_age = 0
        for idx, row in local.iterrows():
            current_viscode = row["VISCODE"]
            local_idx = (test_data["VISCODE"] == current_viscode) & (
                test_data["RID_HASH"] == rid
            )
            if prev_age > 0 and prev_age == prev_age:
                pred_age = (current_viscode - prev_viscode) / 12 + prev_age
            else:
                pred_age = row[col]

            if pred_age == pred_age:  # test not-NaN
                # print("forward imputed", pred_age, current_viscode)
                test_data.loc[local_idx, col] = test_data.loc[local_idx][col].fillna(
                    pred_age
                )

            prev_viscode = row["VISCODE"]
            prev_age = pred_age

        # reverse impute age
        prev_viscode = 0
        prev_age = 0
        for idx, row in local.iloc[::-1].iterrows():
            current_viscode = row["VISCODE"]
            local_idx = (test_data["VISCODE"] == current_viscode) & (
                test_data["RID_HASH"] == rid
            )

            if prev_age > 0 and prev_age == prev_age:
                pred_age = prev_age - (prev_viscode - current_viscode) / 12
            else:
                pred_age = row[col]

            if pred_age == pred_age:  # test not-NaN
                # print("reversed imputed", pred_age, current_viscode)
                test_data.loc[local_idx, col] = test_data.loc[local_idx][col].fillna(
                    pred_age
                )

            prev_viscode = row["VISCODE"]
            prev_age = pred_age

        # print(test_data[(test_data["RID_HASH"] == rid)][["VISCODE", "AGE"]])
    return test_data

## The complete imputation algorithm

__Step 1.__ Impute the constants [`PTGENDER_num`, `PTEDUCAT`, `APOE4`] and the `AGE` from the existing observed data.

__Step 2.__ Longitudinal imputation loop:

        2.1 Create the intermediary imputation using ffill/bfill and HyperImpute. This is just a support for the longitudinal imputers to not deal with the missing values.
        2.2 Run the longitudinal imputation for one step. For each column C with a missing value: if the value from the previous `VISCODE` was observed, we use the __forward__  longitudinal imputer to approximate it. If the value from the next `VISCODE` is observed, we use the __reverse__ longitudinal imputer. If both values are observed, we use both imputers and average their outputs.
        2.3 If no value was imputed in the current step, exit the loop. Else, continue.
        2.4 At this step, the missing features are completely unobserved for a patient. Otherwise, they would have been filled in the longitudinal loop. 
        
__Step 3.__  We run the horizontal imputation to fill in the missing values.For each patient, we merge the horizontal imputation into the rows with the __least missing values__. We are not using the full horizontal imputation because it could miss some temporal patterns. Instead, we use it to seed a second longitudinal imputation loop. In particular, this step imputes all the patients with a single visit.

__Step 4.__ Repeat the constant/AGE imputation with the new seed values from the horizontal imputation.

__Step 5.__ Repeat the longitudinal loop and fill in the remaining missing values.
    

In [18]:
def impute_longitudinal(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    eval_cols: list = [
        "DX_num",
        "CDRSB",
        "MMSE",
        "ADAS13",
        "Ventricles",
        "Hippocampus",
        "WholeBrain",
        "Entorhinal",
        "Fusiform",
        "MidTemp",
    ],
):
    """Longitudinal imputation loop.

    We fill the missing values with intermediary values using ffill/bfill, and HyperImpute, if needed.

    For each patient, and for each missing value from column C:
        - If the current patient has any observed value from column C, we use that for the longitudinal imputer from
        the __forward__ or __reverse__ set(depending on the position related to the missing value).
        - If both previous and next values are observed for a missing value, we use both longitudinal imputers,
        and average their prediction.
    """
    test_data = test_data.copy()
    train_data = train_data.copy()

    imputed_test_data = intermediary_imputation(train_data, test_data)

    train_data = train_data.sort_values(["RID_HASH", "VISCODE"])
    test_data = test_data.sort_values(["RID_HASH", "VISCODE"])
    imputed_test_data = imputed_test_data.sort_values(["RID_HASH", "VISCODE"])

    prev_cols = [f"prev_{col}" for col in train_cols]

    for rid in test_data["RID_HASH"].unique():
        patient = test_data[test_data["RID_HASH"] == rid]
        patient_imputed = imputed_test_data[imputed_test_data["RID_HASH"] == rid]

        prediction_rows = [pd.Series(np.zeros(len(prev_cols)), index=train_cols)]
        for ridx, row in patient.iterrows():
            prediction_rows.append(row[train_cols])
        prediction_rows.append(pd.Series(np.zeros(len(prev_cols)), index=train_cols))

        for col in eval_cols:
            if patient[col].isna().sum() == 0:
                continue

            for ridx, row in enumerate(prediction_rows[1:-1]):
                real_idx = ridx + 1
                if row[col] == row[col]:
                    continue
                current_viscode = row["VISCODE"]
                local_idx = (test_data["VISCODE"] == current_viscode) & (
                    test_data["RID_HASH"] == rid
                )

                prev_col_val = prediction_rows[real_idx - 1][col]
                next_col_val = prediction_rows[real_idx + 1][col]

                if next_col_val == next_col_val and ridx + 1 < len(patient_imputed):
                    eval_data = (
                        patient_imputed.iloc[ridx].to_frame().T[train_cols]
                    ).drop(
                        columns=[col]
                    )  # row.to_frame().T[train_cols]
                    eval_data[prev_cols] = (
                        patient_imputed.iloc[ridx + 1].to_frame().T[train_cols].values
                    )
                    eval_data = eval_data.astype(float)

                    assert eval_data.isna().sum().sum() == 0
                    assert eval_data[f"prev_{col}"].values[0] == next_col_val

                    imputer = longitudinal_imputers["reverse"][col]
                    imputed_val = imputer.predict(eval_data).values.squeeze()

                    test_data.loc[local_idx, col] = imputed_val

                if prev_col_val == prev_col_val and ridx > 0:
                    # print("Imputing using the prev value", prev_col_val)
                    eval_data = (
                        patient_imputed.iloc[ridx].to_frame().T[train_cols]
                    ).drop(columns=[col])
                    eval_data[prev_cols] = (
                        patient_imputed.iloc[ridx - 1].to_frame().T[train_cols].values
                    )
                    eval_data = eval_data.astype(float)

                    assert eval_data.isna().sum().sum() == 0
                    assert eval_data[f"prev_{col}"].values[0] == prev_col_val

                    imputer = longitudinal_imputers["forward"][col]
                    imputed_val = imputer.predict(eval_data).values.squeeze()

                    existing_value = test_data.loc[local_idx, col].values[0]
                    if existing_value == existing_value:
                        imputed_val = (imputed_val + existing_value) / 2
                    test_data.loc[local_idx, col] = imputed_val

    return test_data


def intermediary_imputation(train_data, test_data, forward_first=True):
    """Helper for the longitudinal imputation.
    We use it to fill the missing values using the previous(ffill), or future values(bfill).
    If a feature is completely missing, we fill it using Hyperimpute.

    This imputated dataset it is used only as a backbone for the longitudinal imputation,
    and it is not used in the final results.
    """
    test_data = test_data.copy()

    for rid in test_data["RID_HASH"].unique():
        local = test_data[test_data["RID_HASH"] == rid]

        if forward_first:
            local = local.ffill().bfill()
        else:
            local = local.bfill().ffill()

        test_data.loc[test_data["RID_HASH"] == rid] = local

    test_data = impute_consts(train_data, test_data)
    test_data = impute_age(train_data, test_data)
    return horizontal_imputation(train_data, test_data)


def merge_static_imputation(train_data, test_data, static_imputation):
    """Given a static imputation from HyperImpute, for each patient:
        - We select the row with the least missing values.
        - We impute that row with the values from the static imputation output.

    This is useful for features that are completery missing for a patient.
    We fill only one row, and let the longitudinal imputers fill the rest.
    """
    test_data = test_data.copy()
    train_data = train_data.copy()

    train_data = train_data.sort_values(["RID_HASH", "VISCODE"])
    test_data = test_data.sort_values(["RID_HASH", "VISCODE"])

    for rid in test_data["RID_HASH"].unique():
        patient = test_data[test_data["RID_HASH"] == rid]
        misses = []
        viscodes = []
        for idx, row in patient.iterrows():
            misses.append(row.isna().sum())
            viscodes.append(row["VISCODE"])
        cidx = np.argmin(misses)

        current_viscode = viscodes[cidx]
        local_idx = (test_data["VISCODE"] == current_viscode) & (
            test_data["RID_HASH"] == rid
        )
        imputed_idx = (static_imputation["VISCODE"] == current_viscode) & (
            static_imputation["RID_HASH"] == rid
        )

        if len(test_data[local_idx]) == 0:
            continue

        for col in test_data.columns:
            val = test_data.loc[local_idx][col].values[0]
            if val == val:
                continue
            imputed_val = static_imputation.loc[imputed_idx][col].values[0]
            test_data.loc[local_idx, col] = imputed_val

            # print("imputed", test_data.loc[local_idx, col])

    return test_data


def impute_data(train_data, test_data):
    """Main imputation loop.

    Steps:

    1. Impute constant and AGE features.
    2. Longitudinal imputation loop:
        2.1 Create the intermediary imputation.
        2.2 Run the longitudinal imputation for one step.
        2.3 If no value was imputation in the current step, exit the loop. Else, continue.
    3. Create the static imputation of the remaining missing values.
    4. For each patient, merge the static imputation into the row with the least missing values.
    In particular, this step imputes all the patients with a single visit.
    5. Repeat the constant/AGE imputation with the new seed values from the static imputation.
    6. Repeat the longitudinal loop and fill the remaining missing values.

    """
    test_id = dataframe_hash(test_data)
    train_id = dataframe_hash(train_data)

    print("Evaluate constants", test_id, test_data.isna().sum().sum())
    test_data = impute_consts(train_data, test_data)
    test_data = impute_age(train_data, test_data)

    while True:
        print("Evaluate longitudinals", test_id, test_data.isna().sum().sum())
        new_test_data = impute_longitudinal(train_data, test_data)
        if new_test_data.isna().sum().sum() == test_data.isna().sum().sum():
            break

        test_data = new_test_data

    print(
        "Evaluate static imputation",
        test_id,
        test_data.isna().sum().sum(),
    )
    static_imputation = horizontal_imputation(train_data, test_data)

    test_data = merge_static_imputation(train_data, test_data, static_imputation)

    print("Evaluate constants take 2", test_id, test_data.isna().sum().sum())
    test_data = impute_consts(train_data, test_data)
    test_data = impute_age(train_data, test_data)

    while True:
        print("Evaluate longitudinals take 2", test_id, test_data.isna().sum().sum())
        new_test_data = impute_longitudinal(train_data, test_data)
        if new_test_data.isna().sum().sum() == test_data.isna().sum().sum():
            break

        test_data = new_test_data

    return test_data

In [19]:
from hyperimpute.utils.benchmarks import RMSE


def eval_training_error(train_data, test_data):
    """Helper for benchmarking the imputation"""
    imputed = impute_data(train_data, test_data)
    mask = test_data.isna().astype(int)

    return RMSE(
        imputed.drop(columns=["RID_HASH"]).values,
        train_data.drop(columns=["RID_HASH"]).values,
        mask.drop(columns=["RID_HASH"]).values,
    )


eval_training_error(dev_set, dev_1)

In [20]:
eval_training_error(dev_set, dev_2)

## Submission data

For the final submission data, we concatenate test_A and test_B, and impute them.

This is because both may contain fully observed data, which can be used next to the training set for selecting/training the models.

In [ ]:
test_AB = pd.concat([dev_set, test_A, test_B], ignore_index=True)

test_AB_imputed = impute_data(dev_set, test_AB)
test_AB_imputed[scaled_cols] = scaler.inverse_transform(test_AB_imputed[scaled_cols])

test_AB_imputed

In [ ]:
import numpy as np

results = []


def normalize_output(test_data):
    """Normalize the feature for the final submission."""
    test_data = test_data.copy()

    factor = test_data["CDRSB"] / 0.5
    factor[factor < 0] = 0
    factor = factor.fillna(-1)
    factor = factor.round(0).astype(int)
    factor = factor.replace(-1, np.nan)
    test_data["CDRSB"] = factor * 0.5

    test_data["DX_num"] = test_data["DX_num"].round(0)

    test_data["ADAS13"] = ((test_data["ADAS13"] * 3).round(0) / 3).round(2)
    test_data["MMSE"] = test_data["MMSE"].round(0)

    return test_data


def dump_results(imputed_data: pd.DataFrame, fpath: str):
    """Create the submission file"""
    for name, data in [
        ("test_A", test_A),
        ("test_B", test_B),
    ]:
        for idx, row in data.iterrows():
            for col in row.index:
                local = row.T
                val = local[col]
                if val == val:
                    continue
                imputed_id = f"{local['RID_HASH']}_{local['VISCODE']}_{col}_{name}"
                imputed_val = imputed_data[
                    (imputed_data["RID_HASH"] == local["RID_HASH"])
                    & (imputed_data["VISCODE"] == local["VISCODE"])
                ][col].values[0]

                assert imputed_val == imputed_val
                assert imputed_val != ""

                results.append([imputed_id, imputed_val])

    output = pd.DataFrame(results, columns=submission.columns)
    output.to_csv(fpath, index=None)

    return output


output_normalized = dump_results(
    normalize_output(test_AB_imputed),
    results_dir / f"submission.csv",
)

output_normalized